In [1]:
pip install transformers


In [2]:
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
data_train = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

In [4]:
def clear_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'[^ a-z]', '', text)
    while text.find('  ') != -1:
        text = text.replace('  ', ' ')
    return text

In [5]:
data_train['clean_text']= data_train['Text'].apply(lambda x:clear_text(x))
data_test['clean_text']= data_test['Text'].apply(lambda x:clear_text(x))

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [7]:
data_train['clean_text'] = data_train['clean_text'].apply(lambda x:tokenizer(x))
data_test['clean_text'] = data_test['clean_text'].apply(lambda x:tokenizer(x))

In [8]:
data_train

,Text,Emotion,clean_text
0,i didnt feel humiliated,sadness,"[input_ids, token_type_ids, attention_mask]"
1,i can go from feeling so hopeless to so damned...,sadness,"[input_ids, token_type_ids, attention_mask]"
2,im grabbing a minute to post i feel greedy wrong,anger,"[input_ids, token_type_ids, attention_mask]"
3,i am ever feeling nostalgic about the fireplac...,love,"[input_ids, token_type_ids, attention_mask]"
4,i am feeling grouchy,anger,"[input_ids, token_type_ids, attention_mask]"
...,...,...,...
15993,i most days feel like if braeden and calvin ar...,happy,"[input_ids, token_type_ids, attention_mask]"
15994,i feel so sympathetic empathetic towards them,love,"[input_ids, token_type_ids, attention_mask]"
15995,i just had a very brief time in the beanbag an...,sadness,"[input_ids, token_type_ids, attention_mask]"
15996,i am now turning and i feel pathetic that i am...,sadness,"[input_ids, token_type_ids, attention_mask]"


In [9]:
#good: happy, love, surprise
#bad: sadness, anger, fear

def t_or_f(emotion):
    if emotion == 'happy' or emotion == 'love' or emotion == 'surprise':
        return 1        
    else:        #elif emotioin == 'sadness' or 'anger' or 'fear': return 0
        return 0
    
data_train['Emotion_in_digit'] = data_train['Emotion'].apply(lambda x:t_or_f(x))
data_test['Emotion_in_digit'] = data_test['Emotion'].apply(lambda x:t_or_f(x))

In [10]:
data_train[['Emotion_in_digit', 'Text']]

,Emotion_in_digit,Text
0,0,i didnt feel humiliated
1,0,i can go from feeling so hopeless to so damned...
2,0,im grabbing a minute to post i feel greedy wrong
3,1,i am ever feeling nostalgic about the fireplac...
4,0,i am feeling grouchy
...,...,...
15993,1,i most days feel like if braeden and calvin ar...
15994,1,i feel so sympathetic empathetic towards them
15995,0,i just had a very brief time in the beanbag an...
15996,0,i am now turning and i feel pathetic that i am...


In [11]:
def tokenize(sequence):
    tokens = tokenizer.encode_plus(sequence, max_length=512,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

In [12]:
num_of_elements = 5000
Xids = np.zeros((num_of_elements, 512))
Xmask = np.zeros((num_of_elements, 512))

idx = np.random.randint(0, 10000, num_of_elements)  #was 25000
small_dataset = np.array(data_train['Text'])[idx.astype(int)]
small_dataset_labels = np.array(data_train['Emotion_in_digit'])[idx.astype(int)]

labels = np.array(small_dataset_labels)

In [13]:
for i, sequence in enumerate(small_dataset):
    tokens = tokenize(sequence)
    Xids[i, :], Xmask[i, :] = tokens[0], tokens[1]

In [14]:
 Xids[10]

array([  101.,  1045.,  2001.,  3110.,  2292.,  8167., 12863.,  5292.,
        3270.,  3270.,   102.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
      

In [15]:
from transformers import TFAutoModel

In [16]:
bert = TFAutoModel.from_pretrained('bert-base-uncased')


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [17]:
bert.summary()


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


In [18]:
import tensorflow as tf

input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

embeddings = bert.bert(input_ids, attention_mask=mask)[1]  # we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)

# Classifier head
x = tf.keras.layers.Dense(1024, activation ='relu')(embeddings)
y = tf.keras.layers.Dense(1, activation ='sigmoid', name='outputs')(x)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable = False

In [19]:
bert.bert(input_ids, attention_mask=mask) # outputs of bert


TFBaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                                 <KerasTensor: shape=(None, 512, 768) dtype=float32 (created by layer 'bert')>),
                                                ('pooler_output',
                                                 <KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'bert')>)])

In [20]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [21]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])


In [22]:
history = model.fit(
    [Xids, Xmask], labels,
    validation_split=0.5,
    batch_size = 64,
    verbose = 1,
    epochs=10)

Epoch 1/10
40/40 [==============================] - 410s 10s/step - loss: 0.8076 - accuracy: 0.5552 - val_loss: 0.6408 - val_accuracy: 0.6528
Epoch 2/10
40/40 [==============================] - 392s 10s/step - loss: 0.6323 - accuracy: 0.6492 - val_loss: 0.5981 - val_accuracy: 0.6984
Epoch 3/10
40/40 [==============================] - 392s 10s/step - loss: 0.6250 - accuracy: 0.6424 - val_loss: 0.5776 - val_accuracy: 0.6968
Epoch 4/10
40/40 [==============================] - 392s 10s/step - loss: 0.6084 - accuracy: 0.6800 - val_loss: 0.6292 - val_accuracy: 0.6400
Epoch 5/10
40/40 [==============================] - 392s 10s/step - loss: 0.5805 - accuracy: 0.6924 - val_loss: 0.6129 - val_accuracy: 0.6544
Epoch 6/10
40/40 [==============================] - 392s 10s/step - loss: 0.5672 - accuracy: 0.6984 - val_loss: 0.7442 - val_accuracy: 0.5980
Epoch 7/10
40/40 [==============================] - 392s 10s/step - loss: 0.6200 - accuracy: 0.6548 - val_loss: 0.5789 - val_accuracy: 0.6772
Epoch 